# Discrete Value Iteration
This notebook serves as a tutorial for using the DiscreteValueIteration.jl module, and aims to illustrate:

* How to correctly define the functions used by the module
* How to use the basic value iteration solver
* How to use the parallel value iteration solver
* How to work with the Policy and Solver types provided by the module

This module is tailored to dealing with sparse MDPs that have large state spaces. To that end, it requires the user to define the functions that represent the rewards, and the dynamics of the MDP as opposed to explicitly creating the reward and transition matrices. This tutrial will use a random MDP as an example. A more compelx example can be found in GridWorlds.jl. 

DiscreteValueIteration.jl requires DiscreteMDPs.jl, in which the API for handling discrete MDPs is defined.

## A Quick Note
The easiest way to get the parallel solver to work is to add the following command before importing any of the modules you plan to use.

In [1]:
# first add the number of processors you would like to use in the parallel solver
addprocs(int(CPU_CORES/2)); # CPU_CORES includes hyperthreaded cores, divide by two

2-element Array{Any,1}:
 2
 3

## Defining the MDP
This section defines a random MDP. It uses the functions and abstract types defined in DiscreteMDPs. To avoid @everywhere declarations, wrap your MDP code in a module, and import it after calling addprocs. 

To use the solver, the following functions must be defined:
```julia
states(mdp::YourMDP) # returns an iterator over MDP states
actions(mdp::YourMDP) # returns an iterator over MDP actions
numStates(mdp::YourMDP) # returns the number of states
numActions(mdp::YourMDP) # returns the number of actions
nextStates(mdp::YourMDP, state, action) # returns arrays neighboring states and their probabilities 
reward(mdp::YourMDP, state, action) # returns the immediate reward of the state-action pair
```

The reward function is: $R(s,a)$. The nextStates function is similar to a sparse transition function. For a given pair $(s, a)$, it returns the neighboring states $s'$ and their transition probabilities $T(s'\mid s, a)$



In [2]:
#################################################################
###################### RandomMDPs.jl ############################
#################################################################

# The following is defined in RandomMDPs.jl in the test directory

#=
module RandomMDPs

export RandomMDP
export states, actions
export numStates, numActions
export reward, nextStates

using DiscreteMDPs

import DiscreteMDPs.DiscreteMDP
import DiscreteMDPs.reward
import DiscreteMDPs.nextStates
import DiscreteMDPs.states
import DiscreteMDPs.actions
import DiscreteMDPs.numStates
import DiscreteMDPs.numActions


type RandomMDP <: DiscreteMDP

    nStates::Int64
    nActions::Int64
    nNeighbors::Int64 # numbers of neighbors per state
    randomSeed::Int64

end


function states(mdp::RandomMDP)
    # returns an iterator over the states
    return 1:mdp.nStates
end

function actions(mdp::RandomMDP)
    # returns an iterator over the actions
    return 1:mdp.nActions
end

function numStates(mdp::RandomMDP)
    # returns the number of states
    return mdp.nStates
end

function numActions(mdp::RandomMDP)
    # returns the number of actions
    return mdp.nActions
end


function reward(mdp::RandomMDP, state::Int64, action::Int64)
    # reward is generated using the random seed and the action+state sum
    seed = state+action+mdp.randomSeed
    srand(seed)

    r = rand()
    return r
end


function nextStates(mdp::RandomMDP, state::Int64, action::Int64)
    # returns the nighboring states and their probabilities
    randomSeed = mdp.randomSeed
    nNeighbors = mdp.nNeighbors
    nStates    = numStates(mdp)

    # these can be pre-allocated in the MDP type
    states = zeros(Int64, nNeighbors)
    probs  = zeros(nNeighbors)

    # set a unique seed for each (state, action) pair
    seed = state+action+mdp.randomSeed
    srand(seed)
    for i = 1:nNeighbors
        # random neighbors and probabilities
        states[i] = rand(1:nStates)
        probs[i]  = rand()
    end

    # normalize
    norm = sum(probs)
    for i = 1:nNeighbors
        probs[i] = probs[i] / norm
    end
    return states, probs
end

end # module
=#

## Using the Module

We now have everything we need to run the solver. First, we will run through an example of using the serial solver. If you plan to use the parallel solver, make sure you include the addprocs command before defining or importing the MDP function definitions, and the DiscreteValueIteration module. 

In [3]:
# let's load the module
using DiscreteValueIteration
using RandomMDPs

# create the MDP type
mdp = RandomMDP(500, 5, 5, 1) # input args: state size, action size, neighbors, seed

RandomMDP(500,5,5,1)

## Serial Solver
The serial solver type has a number of optional input arguments:
* maxIterations: the maximum number of iterations in the DP loop (default is 1000)
* tolerance: the Bellman residual (default is 1e-3)
* gaussSiedel: flag for Gauss-Siedel value iteration (default is true)
* includeV: flag for returning the utility function (defualt is true)
* includeQ: flag for returning the Q-matrix (default is true)
* includeA: flag for returning the policy (default is true)

In [4]:
maxIterations = 50 # maximum number of iterations in the DP loop
tolerance = 1e-2   # Bellman residual
gs = true          # Gauss-Siedel falg
includeV = true    # return utility flag
includeQ = true    # return Q-matrix flag
includeA = false   # return policy flag
solver = SerialSolver(maxIterations=maxIterations, tolerance=tolerance, gaussSiedel=gs,
                      includeV=includeV, includeQ=includeQ, includeA=includeA)
solver = SerialSolver(maxIterations=50) # this also works

SerialSolver(50,0.001,true,true,true,true)

In [5]:
policy = solve(solver, mdp); # solve using value iteration

DiscretePolicy{Float64,Float64,Int64}([101.49,101.49,101.49,101.459,101.459,101.413,101.47,101.47,101.47,101.47  …  103.349,103.349,103.378,103.378,103.378,103.378,103.378,103.203,102.971,102.971],5x500 Array{Float64,2}:
 101.407  100.932  101.49   101.047  …  103.378  103.203  102.75   102.971
 100.932  101.49   101.047  101.459     103.203  102.75   102.971  102.187
 101.49   101.047  101.459  100.09      102.75   102.971  102.187  101.759
 101.047  101.459  100.09   101.413     102.971  102.187  101.759  102.432
 101.459  100.09   101.413  100.22      102.187  101.759  102.432  102.966,[3,2,1,2,1,2,5,4,3,2  …  3,2,5,4,3,2,1,1,2,1])

The following API gives access to utility, Q-matrix and policy values:

In [6]:
s = 1
a = 1
u  = value(policy, s) # expected optimal value for state s
q  = value(policy, s, a) # expected value for state-action pair
ap = action(policy, s) # action that maximizes the expected utility

3

## Parallel Solver
The parallel solver has two additional input arguments:
* numProcessors: the number of processors to be used by the solver (this argument is required)
* stateOrdering: used for backwards induction value iteration to indicate the state ordering (optinal argument)
The stateOrder is an Array of UnitRange types (iterators) over the states that need to be updated before other states. In the example below, the 1:250 chunk will be updated before the 251:500 chunk at each iteration.

In [7]:
numProcs   = 2 # numProcs should not exceed the value returned by nprocs()
stateOrder = {1:250,251:500} # default ordering is {[1,numStates]}
solver = ParallelSolver(numProcs, stateOrder=stateOrder, maxIterations=maxIterations,
                        tolerance=tolerance, gaussSiedel=gs,
                        includeV=includeV, includeQ=includeQ, includeA=includeA)
solver = ParallelSolver(numProcs, maxIterations=50) # this also works

ParallelSolver(2,None[],50,0.001,true,true,true,true)

In [8]:
policy = solve(solver, mdp)

DiscretePolicy{Float64,Float64,Int64}([101.49,101.49,101.49,101.459,101.459,101.413,101.47,101.47,101.47,101.47  …  103.349,103.349,103.378,103.378,103.378,103.378,103.378,103.203,102.971,102.971],5x500 SharedArray{Float64,2}:
 101.407  100.932  101.49   101.047  …  103.378  103.203  102.75   102.971
 100.932  101.49   101.047  101.459     103.203  102.75   102.971  102.187
 101.49   101.047  101.459  100.09      102.75   102.971  102.187  101.759
 101.047  101.459  100.09   101.413     102.971  102.187  101.759  102.432
 101.459  100.09   101.413  100.22      102.187  101.759  102.432  102.966,[3,2,1,2,1,2,5,4,3,2  …  3,2,5,4,3,2,1,1,2,1])

The same API can be used to access the utility, Q-matrix, and policy values:

In [9]:
s = 1
a = 1
u  = value(policy, s) # expected optimal value for state s
q  = value(policy, s, a) # expected value for state-action pair
ap = action(policy, s) # action that maximizes the expected utility

3